In [118]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

In [ ]:
df1 = dd.read_csv("Tax Records\\Liens.csv")
df2 = dd.read_csv("Tax Records\\Property Assessments Parcel Data.csv")
df3 = dd.read_csv("Tax Records\\Property Currently with Tax Liens.csv")
df4 = dd.read_csv("Tax Records\\Satified Liens.csv")

In [122]:
# Merge the DataFrames on the specified columns
merged_df = dd.merge(df2, df4, left_on='PARID', right_on='pin', how='inner')

# Display the merged DataFrame (compute to get a Pandas DataFrame)
print(merged_df.compute())

         _id_x             PARID  PROPERTYHOUSENUM  PROPERTYFRACTION  \
0           27  0001D00150000000             130.0            -160.0   
1           27  0001D00150000000             130.0            -160.0   
2           27  0001D00150000000             130.0            -160.0   
3           27  0001D00150000000             130.0            -160.0   
4           27  0001D00150000000             130.0            -160.0   
...        ...               ...               ...               ...   
553844  583978  0131M00079000000             131.0               NaN   
553845  583978  0131M00079000000             131.0               NaN   
553846  583978  0131M00079000000             131.0               NaN   
553847  583978  0131M00079000000             131.0               NaN   
553848  584005  0801A00355000000             805.0               NaN   

       PROPERTYADDRESS PROPERTYCITY PROPERTYSTATE PROPERTYUNIT  PROPERTYZIP  \
0              5TH AVE   PITTSBURGH            PA       

In [123]:
# Merge the DataFrames on the specified columns
merged_df1 = dd.merge(df1, df3, left_on='_id', right_on='_id-2', how='inner')

# Display the merged DataFrame (compute to get a Pandas DataFrame)
print(merged_df1.compute())

         _id_x             pin_x  number_x  total_amount   _id_y  \
0      3510673  0001B00010000000        11      13585.47       6   
1      3510674  0001C00058000000         2       7827.64     915   
2      3510675  0001C01661000000         6      14038.90     919   
3      3510676  0001C016610W2100         1        805.33     910   
4      3510677  0001C01661W13700         3       3654.30    1007   
...        ...               ...       ...           ...     ...   
52744  3597489  9952X50391000000       161      29439.48  485703   
52745  3597490  9952X50445000000       135      27348.74  485704   
52746  3597491  9952X50624000000       102        953.80  485705   
52747  3597492  9952X50993000000       153       2290.49  485706   
52748  3597493  9952X83034000000        18       5359.65  485708   

                  pin_y  Street Address Number  Street Address Line 2  \
0      0001B00010000000                    601                    NaN   
1      0001C00058000000              

In [126]:
# Drop unwanted columns from merged_df
merged_df = merged_df.drop(columns=['dtd', 'ward', 'Unit or Apartment Number', 'assignee'], errors='ignore')

# Drop unwanted columns from merged_df1
merged_df1 = merged_df1.drop(columns=['dtd', 'ward', 'Unit or Apartment Number', 'assignee'], errors='ignore')

# Convert PROPERTYZIP to string (if you want to keep leading zeros)
merged_df['PROPERTYZIP'] = merged_df['PROPERTYZIP'].astype(str)

# Convert Zip Code to string (if you want to keep leading zeros)
merged_df1['Zip Code'] = merged_df1['Zip Code'].astype(str)

# Merge the DataFrames on the specified columns with different names
final_merged_ddf = dd.merge(
    merged_df,
    merged_df1,
    left_on=['PROPERTYCITY', 'PROPERTYZIP'],  # Columns from merged_df
    right_on=['City', 'Zip Code'],            # Columns from merged_df1
    how='outer'                                # Keep all data
)

# Display the final merged DataFrame (compute to get a Pandas DataFrame)
print(final_merged_ddf.compute())

         _id_x_x             PARID  PROPERTYHOUSENUM  PROPERTYFRACTION  \
0        70272.0  0016J00007000000            1001.0               NaN   
1            NaN              <NA>               NaN               NaN   
2            NaN              <NA>               NaN               NaN   
3            NaN              <NA>               NaN               NaN   
4            NaN              <NA>               NaN               NaN   
...          ...               ...               ...               ...   
606593  232562.0  0180L00161000000             125.0               NaN   
606594  135554.0  0307S00364000000               0.0               NaN   
606595  135554.0  0307S00364000000               0.0               NaN   
606596  135554.0  0307S00364000000               0.0               NaN   
606597  135554.0  0307S00364000000               0.0               NaN   

       PROPERTYADDRESS PROPERTYCITY PROPERTYSTATE PROPERTYUNIT PROPERTYZIP  \
0           E ENTRY DR        152

In [130]:
# Count the number of entries for each Municipality Name
municipality_counts = final_merged_ddf['Municipality Name'].value_counts().compute()
municipality_counts 

Municipality Name
10th Ward -  McKEESPORT     260
10th Ward - PITTSBURGH      671
11th Ward  - McKEESPORT     146
11th Ward - PITTSBURGH      287
12th Ward  - McKEESPORT     124
                           ... 
White Oak                   378
Whitehall                   138
Wilkins                     524
Wilkinsburg                1554
Wilmerding                  141
Name: count, Length: 175, dtype: int64[pyarrow]